In [1]:
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from skimage.io import imread
from tqdm.notebook import tqdm
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [4]:
dir_path = '../data/'

train, train_labels = [], []
with open(dir_path + 'train.txt', 'r') as f:
  for line in f:
    data = line.split(" ")
    train.append(dir_path + data[0])
    train_labels.append(data[1].replace('\n', ''))


val, val_labels = [], []
with open(dir_path + 'val.txt', 'r') as f:
  for line in f:
    data = line.split(" ")
    val.append(dir_path + data[0])
    val_labels.append(data[1].replace('\n', ''))


test, test_labels = [], []
with open(dir_path + 'test.txt', 'r') as f:
  for line in f:
    data = line.split(" ")
    test.append(dir_path + data[0])
    test_labels.append(data[1].replace('\n', ''))

print(f'訓練資料共{len(train)}筆')
print(f'驗證資料共{len(val)}筆')
print(f'測試資料共{len(test)}筆')

訓練資料共63325筆
驗證資料共450筆
測試資料共450筆


In [ ]:
def feature_extraction(paths):
    features = []
    for path in tqdm(paths):
        image = imread(path, 0) 
        # 灰階圖片數值介於0~255之間
        image_normalize = image/255
        # 將圖片壓縮成64*80
        image_resize = np.resize(image_normalize, (64, 64, 1))
        features.append(image_resize)          
    return np.array(features)

In [ ]:
# 資料前處理
train_features = feature_extraction(train)
train_labels_oneHot = np_utils.to_categorical(train_labels)
val_features = feature_extraction(val)
val_labels_oneHot = np_utils.to_categorical(val_labels)
test_features = feature_extraction(test)
test_labels_oneHot = np_utils.to_categorical(test_labels)

In [ ]:
model = Sequential()
# 兩層卷積層+最大池化層
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#drop掉部分神經元避免overfitting
model.add(Dropout(0.5))
model.add(Flatten()) #平坦化
model.add(Dense(64, activation='relu'))
model.add(Dense(len(train_labels_oneHot[0]), activation='softmax'))
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
# x_train, x_test, y_train, y_test = train_test_split(features, train_labels_oneHot, test_size=0.2, random_state=0)
train_history=model.fit(train_features, train_labels_oneHot, epochs=10, batch_size=300, verbose=1, validation_data=(val_features, val_labels_oneHot))

In [ ]:
def plot_my_accuracy(train_acc,test_acc):
    plt.plot(train_history.history[train_acc])
    plt.plot(train_history.history[test_acc])
    plt.title('Accuracy History')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

plot_my_accuracy('accuracy','val_accuracy')

def plot_my_loss(loss,val_loss):
    plt.clf()
    plt.plot(train_history.history[loss])
    plt.plot(train_history.history[val_loss])
    plt.title('Loss History')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

plot_my_loss('loss','val_loss')

In [ ]:
test_labels=model.predict(test_features, verbose=1, batch_size=32)
loss, accuracy = model.evaluate(test_features, test_labels_oneHot)
print('Test:')
print('Loss: %s\nAccuracy: %s' % (loss, accuracy))